In [1]:
import gzip
import json

def fix_json_str(js):
    t = "[" + js
    t = t.replace("\n", ",")
    t = t[:-1]
    t += "]"
    return t

In [2]:
!pip install pydash

  Using cached pydash-5.1.0-py3-none-any.whl (84 kB)


In [3]:
def format_number(number):
    if number < 10:
        return "00" + str(number)
    elif number < 100:
        return "0" + str(number)
    else:
        return str(number)

In [4]:
import gc
gc.collect()

0

In [5]:
import json
import re
from pydash import get


alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

def get_first_sent(abst):
    return get(split_into_sentences(abst), 0)

In [6]:
import time
import pandas as pd

In [ ]:
data = []
errors = []
start=1000
end = 1300
for i in range(start,end):
    path = 'downloads/s2c/s2-corpus-{}.gz'.format(format_number(i))
    try:
        with gzip.open(path, 'r') as f:
            if (i % 20) == 0 and len(data):
                print("zip: ", i, " t ", time.time())

                df = pd.DataFrame(data)
                df["text"] = df["paperAbstract"].map(get_first_sent)
                df = df[["text", "s2Url", "doi", "journalName", "title", "fieldsOfStudy"]]
    #             df = df.drop(columns=["paperAbstract"])
                df.to_csv("downloads/s2c_{}_{}.csv".format(start, end), index = False, header=True)
            json_bytes = fix_json_str(f.read().decode('utf-8'))
            data += json.loads(json_bytes)
    except:
        print("ERROR:", path)
        errors += path
        

zip:  1020  t  1655374066.3920557
zip:  1040  t  1655374207.6568654
zip:  1060  t  1655374434.8502991
zip:  1080  t  1655374754.9983103
zip:  1100  t  1655375154.2322311
zip:  1120  t  1655375622.8178139
zip:  1140  t  1655376193.4904313
zip:  1160  t  1655376846.3622072


In [ ]:
import pandas as pd

In [17]:
df1 = pd.read_csv("downloads/s2c_0_300.csv",  lineterminator='\n')


In [16]:
df2 = pd.read_csv("downloads/s2c_180_340.csv", lineterminator='\n' )

In [22]:
df= df1.append(df2)

In [28]:
df.to_csv("downloads/s2c_11M_short.csv", index = False, header=True)

In [29]:
!rm -rf cache_dir/

In [30]:
df = df[df.fieldsOfStudy.notnull()]

df["fields"] = df.fieldsOfStudy.map(lambda a: get(eval(a), 0))

In [31]:
df = df[df.fields.isin(["Medicine", 'Environmental Science', 'Sociology', 'Psychology', 'Geology', 'Biology', 'Materials Science', 'Chemistry'])]

In [34]:
df.to_csv("downloads/s2c_5M_bioadjacent.csv",index = False, header=True)

In [29]:
!pip install pydash

  Using cached pydash-5.1.0-py3-none-any.whl (84 kB)


In [31]:

from pydash import get

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,id,title,paperAbstract,authors,inCitations,outCitations,year,s2Url,sources,pdfUrls,...,journalVolume,journalPages,doi,doiUrl,pmid,fieldsOfStudy,magId,s2PdfUrl,entities,fields
0,33b237709dbd53953a750355115b57ccb6690da1,USING PHOTO IMAGES TO DETERMINE THE GEOMETRIC ...,NaN,"[{'name': 'E Popov', 'ids': ['39390170'], 'st...",[],[],2019,https://semanticscholar.org/paper/33b237709dbd...,[],[],...,NaN,NaN,10.14529/build190307,https://doi.org/10.14529/build190307,NaN,['Computer Science'],3.00019e+09,NaN,[],Computer Science
1,062e9c7579adc73129e1198671d05905f07d4ab5,Clinical or Industrial Pharmacy? Case Studies ...,Automated medication dispensing systems for ho...,"[{'name': 'J Novek', 'ids': ['47586327'], 'st...","['663b3c9bbc9a1309d636044e50347e177da74624', '...","['c641b685483c831578dc4cb12e9ff9a68e9751e3', '...",1998,https://semanticscholar.org/paper/062e9c7579ad...,['Medline'],[],...,28,445 - 465,10.2190/W2BT-FGXQ-QL0G-YNL9,https://doi.org/10.2190/W2BT-FGXQ-QL0G-YNL9,9711475.0,['Medicine'],1.98838e+09,NaN,[],Medicine
2,771723e7e2d9ed97d09442086553b726f0dd6657,Noncoronary and coronary atherothrombotic plaq...,"In the future, the use of imaging methods to q...","[{'name': 'Zahi A Fayad', 'ids': ['1886137'], ...","['0de4efab5980b5245d979d4efa21c93c49741c5e', '...","['47af1c2bfd5f65dad41dd64a8ef1618aabcaa233', '...",2002,https://semanticscholar.org/paper/771723e7e2d9...,['Medline'],[],...,12 3,\n 461-71\n,10.1016/S1052-5149(02)00023-0,https://doi.org/10.1016/S1052-5149%2802%2900023-0,12486833.0,['Medicine'],1.98318e+09,NaN,[],Medicine
3,263f70167ba7def67a2843ffc30c80a0b9b3f617,국어 명사구 접속의 연구,NaN,"[{'name': '최재희', 'ids': ['115087709'], 'struct...",[],[],1985,https://semanticscholar.org/paper/263f70167ba7...,[],[],...,NaN,NaN,NaN,NaN,NaN,[],2.29156e+09,NaN,[],None
4,9ea1148fa657103879ef7ed9a5b716be48ee13a4,Who'll Take My Place : When I'm Gone,NaN,"[{'name': 'Billy Fazioli', 'ids': ['146257415...",[],[],1922,https://semanticscholar.org/paper/9ea1148fa657...,[],['https://digitalcommons.library.umaine.edu/cg...,...,NaN,NaN,NaN,NaN,NaN,['Art'],1.48919e+09,NaN,[],Art
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391443,7b4c2627e173c7a7856fca9372f4eef161c98398,Leverage,NaN,"[{'name': 'Desmond McNeill', 'ids': ['1769115...",[],[],2019.0,https://semanticscholar.org/paper/7b4c2627e173...,[],[],...,NaN,NaN,10.4324/9780429292262-3,https://doi.org/10.4324/9780429292262-3,NaN,[],NaN,NaN,[],None
391444,23c319ad766ff431601fe8465d096e152a026ad1,Optical system with sliding lens for ophthalmo...,"FIELD: physics, optics. SUBSTANCE: invention r...","[{'name': 'Ференц Ракши', 'ids': ['95117964']...",[],[],2010.0,https://semanticscholar.org/paper/23c319ad766f...,[],[],...,NaN,NaN,NaN,NaN,NaN,['Engineering'],2750527308,NaN,[],Engineering
391445,462e9ec399188071342ef4692002f072c12b6437,Qualitätspolitik und Qualitätswirtschaft,Wenn unter Unternehmenspolitik das bewusste Du...,"[{'name': 'h.c. Wolfgang Lücke', 'ids': ['1162...",[],[],2003.0,https://semanticscholar.org/paper/462e9ec39918...,[],[],...,NaN,183-184,10.1007/978-3-322-81585-9_8,https://doi.org/10.1007/978-3-322-81585-9_8,NaN,['Political Science'],112768712,NaN,[],Political Science
391446,9746f6bec5ce97bc94bbbfd007443c25992cf3e4,The effect of HNO3 gas on the lichen Ramalina ...,"Abstract Nitric acid (HNO3) and ozone (O3), se...","[{'name': 'Jennifer Riddell', 'ids': ['500267...","['fa4fed37c2fe30f831e05ca701bb759842e15a2c', '...","['c0ac5418337832b96746e1677cb6fa722f73505f', '...",2008.0,https://semanticscholar.org/paper/9746f6bec5ce...,[],['https://naldc.nal.usda.gov/download/17971/PDF'],...,203,47-54,10.1016/J.FLORA.2007.10.001,https://doi.org/10.1016/J.FLORA.2007.10.001,NaN,['Environmental Science'],2005248456,NaN,[],Environmental Science


In [43]:
chem = samp[samp["fieldsOfStudy"].str.contains("Chemistry")]

In [46]:
#env_science

In [44]:
chem

,id,title,paperAbstract,authors,inCitations,outCitations,year,s2Url,sources,pdfUrls,...,journalVolume,journalPages,doi,doiUrl,pmid,fieldsOfStudy,magId,s2PdfUrl,entities,fields
16,8f74c75b5893b163a19c8321746be3c05a06b2db,Influences nutritionnelles et hormonales chron...,NaN,"[{'name': 'Willy Malaisse', 'ids': ['20597062...","['267fb6bd94424bfcd025e57ab7be5f1447e776e3', '...",[],1969,https://semanticscholar.org/paper/8f74c75b5893...,[],[],...,NaN,131-138,NaN,NaN,NaN,['Chemistry'],7.80614e+08,NaN,[],Chemistry
33,a16710c46a617ab5324ef9920eafe104647769a9,[Influence of adrenal in muscle phosphorylase ...,NaN,"[{'name': 'E VARGA', 'ids': ['145060989'], 's...",[],[],1959,https://semanticscholar.org/paper/a16710c46a61...,['Medline'],[],...,11 1,\n 89-96\n,NaN,NaN,13655505.0,"['Chemistry', 'Medicine']",2.46244e+09,NaN,[],Chemistry
46,33a4a5a1597ee2d6a23504dcd671036aba34db59,IN SILICO DOCKING STUDIES ON THE ANTI-CANCER E...,Objective: The statistics on cancer imposes th...,"[{'name': 'Gajendran Nithya', 'ids': ['484475...","['6478fac1bc7a773dc9644d1bb7ca77e95c40f52e', '...",[],2015,https://semanticscholar.org/paper/33a4a5a1597e...,[],[],...,8,192-195,NaN,NaN,NaN,['Chemistry'],2.99421e+09,NaN,[],Chemistry
50,85b10d1c4fb0ce049957d68d098fe791530caf07,[Effect of insulin on the expression of platel...,OBJECTIVE\nTo investigate the effects of insul...,"[{'name': 'Bi-you Peng', 'ids': ['32244875'],...",[],[],2003,https://semanticscholar.org/paper/85b10d1c4fb0...,['Medline'],[],...,23 9,\n 943-5\n,NaN,NaN,13129728.0,"['Chemistry', 'Medicine']",2.38682e+09,NaN,[],Chemistry
57,dea715eaed17c1c0c2f297efb59563847d5db544,High Temperature Sensitivity Pressure Sensors ...,NaN,"[{'name': 'Congcong Yang', 'ids': ['210943966...","['2ed02534dc39c884ca1231da423456bdf1e8de02', '...","['397ecc1775904134717465e546c9ca57376c7685', '...",2019,https://semanticscholar.org/paper/dea715eaed17...,[],[],...,NaN,NaN,10.1149/2.0241914jes,https://doi.org/10.1149/2.0241914jes,NaN,['Chemistry'],2.976e+09,NaN,[],Chemistry
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391383,3f0e63ba0b718eb20f4d82b90534b21dafc5d2b6,1207 分子軌道法による (CO_2+H_2O) の CH_3OH へのプラズマ直接改質プ...,NaN,"[{'name': '加藤 征三', 'ids': ['84683364'], 'stru...",[],[],2000.0,https://semanticscholar.org/paper/3f0e63ba0b71...,[],[],...,2000,211,NaN,NaN,NaN,['Chemistry'],434239575,NaN,[],Chemistry
391385,8f8f60d736c76d20b078ac4d3663c4eb7b8ecdfa,Theoretical and experimental study on Mg(BH4)2...,Abstract After a screening of possible systems...,"[{'name': 'Elisa Albanese', 'ids': ['46837339...","['9282cc3d8f37d4a7935fb198f7db7c7a9dc06261', '...","['d4902cfa276ce15a97e9062035ac498c158b026b', '...",2013.0,https://semanticscholar.org/paper/8f8f60d736c7...,[],['https://brage.bibsys.no/xmlui/bitstream/hand...,...,580,NaN,10.1016/J.JALLCOM.2013.02.165,https://doi.org/10.1016/J.JALLCOM.2013.02.165,NaN,['Chemistry'],1998645953,NaN,[],Chemistry
391389,33b08b1cd49187f211f065435f1f6fc8e4be43a8,Effect of peptide and nonpeptide antagonists o...,Angiotensin II (Ang II) administered intracere...,"[{'name': 'Stefanka Stancheva', 'ids': ['2744...","['e0ea446017326a41996dd81a7bb5c382b4579757', '...","['f8996643bb890ae5e96102e4553dc2f7f1f471a1', '...",2009.0,https://semanticscholar.org/paper/33b08b1cd491...,['Medline'],['http://rabbit.if-pan.krakow.pl/pjp/pdf/2009/...,...,61 6,\n 1206-10\n,10.1016/S1734-1140(09)70185-9,https://doi.org/10.1016/S1734-1140%2809%2970185-9,20081258.0,"['Chemistry', 'Medicine']",2014065664,NaN,[],Chemistry
391404,51132e689b3cf9084dba486382e24ca6801223d2,Differential susceptibility to experimental gl...,The purpose of this experiment was to test the...,"[{'name': 'Frances E Cone', 'ids': ['5149896']...","['24d4b318cfdefb5c32edc133f6bac625dcd8e13a', '...","['66544a1a8f9b8f6ed9a56c46703c8a3cd27605a3', '...",2010.0,https://semanticscholar.org/paper/51132e689b3c...,['Medline'],[],...,91 3,\n 415-24\n,10.1016/j.exer.2010.06.018,https://doi.org/10.1016/j.exer.2010.06.018,20599961.0

In [26]:
field = "Psychology"

In [27]:
sub = [d for d in data if field in d["fieldsOfStudy"]]
len(sub)

42432

In [28]:
import pandas as pd
df = pd.DataFrame(sub)


42432

In [59]:
df2["fields"] = samp.fieldsOfStudy.map(lambda a: get(eval(a), 0))

In [60]:
df=df2

In [61]:
df=df.dropna(subset=["paperAbstract"])
df=df.dropna(subset=["doi"])

df = df[df["paperAbstract"].str.len() > 16]
len(df)

1956472

In [62]:
df["text"] = df["paperAbstract"].map(get_first_sent)

In [63]:
dfx = df[["text", "s2Url", "doi", "journalName", "title", "fields", "fieldsOfStudy"]]

In [65]:
len(dfx)

1956472

In [64]:
dfx.to_csv ("downloads/s2c_100_2M.csv",index = False, header=True)

In [34]:
FoS = []
for d in data:
    FoS += d["fieldsOfStudy"]

print(set(FoS))

{'Business', 'Mathematics', 'Philosophy', 'Medicine', 'Environmental Science', 'Engineering', 'History', 'Art', 'Sociology', 'Psychology', 'Geology', 'Economics', 'Biology', 'Materials Science', 'Physics', 'Geography', 'Chemistry', 'Political Science', 'Computer Science'}
